In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
import re

In [10]:
jumlah_index = 50
threads_link = []
links = []
results = []
threads = []

In [11]:
def scrape_links(page_number):
    url = f"https://www.cnbcindonesia.com/indeks/{page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article')
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [12]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 10 links from page 5
Scraped 10 links from page 16
Scraped 10 links from page 28
Scraped 10 links from page 6
Scraped 10 links from page 8
Scraped 10 links from page 3
Scraped 10 links from page 25
Scraped 10 links from page 37
Scraped 10 links from page 27Scraped 10 links from page 29
Scraped 10 links from page 7
Scraped 10 links from page 30

Scraped 10 links from page 45
Scraped 10 links from page 35
Scraped 10 links from page 17
Scraped 10 links from page 10
Scraped 10 links from page 24
Scraped 10 links from page 4
Scraped 10 links from page 22
Scraped 10 links from page 11
Scraped 10 links from page 1
Scraped 10 links from page 39
Scraped 10 links from page 21
Scraped 10 links from page 38
Scraped 10 links from page 34
Scraped 10 links from page 12
Scraped 10 links from page 41
Scraped 10 links from page 19
Scraped 10 links from page 42
Scraped 10 links from page 49
Scraped 10 links from page 32
Scraped 10 links from page 14
Scraped 10 links from page 13
Scraped 10 links 

In [13]:
def scrape_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        
        # Judul Berita
        title_elem = soup.find('h1')
        if title_elem:
            title_text = title_elem.text.strip()
        else:
            title_text = "Title not found"
        # Author berita
        author_elem = soup.find('div', {"class": "author"})
        if author_elem:
            author_text = author_elem.get_text()
            author_text = author_text.split('-')[1].strip()
            author_text = author_text.split(',')[0].strip()
        else:
            author_text = "Author not found"     
        # tanggal berita
        date_elem = soup.find('div', {"class": "date"})
        if date_elem:
            date_text = date_elem.text.strip()
            datetime_object = datetime.strptime(date_text, '%d %B %Y %H:%M')
            formatted_date = datetime_object.strftime('%Y-%m-%d')
        else:
            date_text = "Date not found"
        #     # Category berita
        category_elements = soup.find('ul', {"class": "breadcrumb"})
        if category_elements:
            category_text = category_elements.find_all('li')
            category_text= category_text[2].get_text()
        else:
            category_text = "Category not found"
        #     # Content Berita
        body_elem = soup.find('div', {"class": "detail_text"})
        
        if body_elem:
            content_elem = body_elem.find_all('p')
            content_text = ""
            for p in content_elem:
                content_text += p.text.strip() + "\n"
            
            if content_text.strip():
                content_text=content_text
            else:
                content_text ="Content not found"
        else:
          content_text ="Content not found"
          
        nama_berita_match = re.search(r'https://www\.(\w+)\.com/', url)
        if nama_berita_match:
            nama_berita = nama_berita_match.group(1)
        else:
             nama_berita = "Nama_berita not found"        

        results.append({'title': title_text,
                        'author' : author_text,
                        'category':category_text,
                        'date': formatted_date,
                        'content' : content_text,
                        'nama_berita' : nama_berita,
                        'link' : url})
    else:
        print(f"Failed to retrieve data from {url}")

In [14]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

Exception in thread Thread-418 (scrape_url):
Traceback (most recent call last):
  File "/home/krisna/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py", line 715, in urlopen
    httplib_response = self._make_request(
                       ^^^^^^^^^^^^^^^^^^^
  File "/home/krisna/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py", line 404, in _make_request
    self._validate_conn(conn)
  File "/home/krisna/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py", line 1058, in _validate_conn
    conn.connect()
  File "/home/krisna/anaconda3/lib/python3.11/site-packages/urllib3/connection.py", line 419, in connect
    self.sock = ssl_wrap_socket(
                ^^^^^^^^^^^^^^^^
  File "/home/krisna/anaconda3/lib/python3.11/site-packages/urllib3/util/ssl_.py", line 449, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
               ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/krisna/anaconda3/lib/python3.11/site-packages/urllib3/util/ssl_.py", li

In [15]:
df = pd.DataFrame(results)

df.head(10)

,title,author,category,date,content,nama_berita,link
0,UMKM EXPO(RT) BRILIANPRENEUR 2023 Hasilkan Kom...,Teti Purwanti,Berita Entrepreneur,2023-12-11,"Jakarta, CNBC Indonesia - UMKM EXPO(RT) BRILIA...",cnbcindonesia,https://www.cnbcindonesia.com/entrepreneur/202...
1,"Korban Teriak! Tagihan Rp11 T, Wanaartha Cuma ...",Mentari Puspadini,Berita Market,2023-12-11,"Jakarta, CNBC Indonesia - Nasabah PT Asuransi ...",cnbcindonesia,https://www.cnbcindonesia.com/market/202312111...
2,"Video: Awal Pekan, IHSG & Rupiah Ditutup Ambruk",CNBC Indonesia TV,Video Market,2023-12-11,"Jakarta, CNBC Indonesia - Indeks Harga Saham G...",cnbcindonesia,https://www.cnbcindonesia.com/market/202312111...
3,"3 Rahasia Jepang Agar Bisnis Tak Bangkrut, Awe...",Muhammad Fakhriansyah,Berita Entrepreneur,2023-12-11,"Jakarta, CNBC Indonesia - Dalam berbisnis sala...",cnbcindonesia,https://www.cnbcindonesia.com/entrepreneur/202...
4,"Tokopedia-TikTok Shop Bersatu, Ini Dampak ke E...",Novina Putri Bestari,Berita Tech,2023-12-11,"Jakarta, CNBC Indonesia - GoTo melalui unit e-...",cnbcindonesia,https://www.cnbcindonesia.com/tech/20231211154...
5,"Ngeri! Israel, AS & China Saingan Rilis Drone ...",Tim Redaksi,Berita Tech,2023-12-10,"Jakarta, CNBC Indonesia - Di tengah perang Isr...",cnbcindonesia,https://www.cnbcindonesia.com/tech/20231210115...
6,Daftar iPhone Banting Harga di RI Desember 202...,Intan Rakhmayanti Dewi,Berita Tech,2023-12-11,"Jakarta, CNBC Indonesia - Akhir tahun menjadi ...",cnbcindonesia,https://www.cnbcindonesia.com/tech/20231211153...
7,"Gak Cuma Cari Cuan, Ini Janji Anies Buat BUMN",Mentari Puspadini,Berita Market,2023-12-11,"Jakarta, CNBC Indonesia - Calon Presiden Nomor...",cnbcindonesia,https://www.cnbcindonesia.com/market/202312111...
8,Kinerja Saham Bank Big Caps,,Audio Paham: Pantauan Saham,2023-12-11,"Hai, Sobat Cuan..\n\nSaham perbankan masih men...",cnbcindonesia,https://www.cnbcindonesia.com/cuap-cuap-cuan/2...
9,Dua KEK Ini Terbukti Jadi Booster Ekonomi Indo...,Revo M,Berita Research,2023-12-10,"Jakarta, CNBC Indonesia - Ada banyak strategi ...",cnbcindonesia,https://www.cnbcindonesia.com/research/2023121...


Save File

In [16]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'../tempat_simpan_nasional/cnbc_data_baru_{current_datetime}.xlsx'
df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')

Data has been saved to ../tempat_simpan_nasional/cnbc_data_baru_2023-12-11_20-31-01.xlsx
